In [5]:
import tweepy
from tweepy import OAuthHandler

# Read csv file into a pandas dataframe
ACCESS_TOKEN = ''
ACCESS_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)

In [11]:
import json
from tweepy import Stream
from tweepy.streaming import StreamListener

class MyListener(StreamListener):
    
    def on_data(self, data):
        try:
            with open('tweetcrawled.json','a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    
    def on_error(self, status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#kamisbucin'])

KeyboardInterrupt: 

In [12]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']

['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


In [16]:
a = 0
with open('tweetcrawled.json', 'r') as f:
    for line in f:
        a +=1
        if a%2!=0:
            tweet = json.loads(line)
            tokens = preprocess(tweet['text'])
            print(tokens)

['#kamisbucin']
['#kamisbucin', '?', '?', 'Bucinannya', 'gaada']
['RT', '@ChintyaAndana', ':', 'Gmna', 'gmna', 'gaes', '?', '#kamisbucin', 'https://t.co/DyYr6ZprzD']
['Belum', 'selesai', 'kegundahan', 'hati', 'karena', 'semalam', 'ditinggal', 'tidur', 'oleh', 'sang', 'pujaan', ',', 'sekarang', 'malah', 'dengar', 'stasiun', 'tv', 'kes', '…', 'https://t.co/2DIwlZDqZl']
['@Kisahanakindie', 'Rinjani', '.', 'resiko', 'jadi', 'bucin', 'masakini', '#kamisbucin']
['RT', '@GagahPriyo', ':', 'GODFATHER', 'OF', 'BROKEN', 'HEART', 'X', 'BEKONANG', '#kamisbucin', 'https://t.co/EIVGJ0RGWh']
['Ingin', 'Tampil', 'Keren', ',', 'Dua', 'Remaja', 'Ini', 'Nekat', 'Berdiri', 'di', 'Samping', 'Kereta', 'Api', ',', 'Hal', 'Ini', 'yang', 'Terjadi', 'Pada', 'Keduanya', '.', '.', '.', '…', 'https://t.co/rMbJFKCwSs']
['HASIL', 'RESULT', 'SYDNEY', 'POOLS', '09', 'AGUSTUS', '2019', '1577', 'SHIO', ':', 'KAMBING', '#JumatBerkah', '#DoaPagiIni', '#kamisbucin', '…', 'https://t.co/bFaBrcizqR']
['Apa', 'cuman', 'gw', 'y

In [18]:
import operator 
import json
from collections import Counter

fname = 'tweetcrawled.json'
with open(fname, 'r') as f:
    count_all = Counter()
    a = 0
    for line in f:
        a +=1
        if a%2!=0:
            tweet = json.loads(line)
            # Create a list with all the terms
            terms_all = [term for term in preprocess(tweet['text'])]
            # Update the counter
            count_all.update(terms_all)
    # Print the first 5 most frequent words
    print(count_all.most_common(5))

[('#kamisbucin', 17), ('.', 13), (':', 9), ('RT', 7), (',', 6)]


In [23]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + stopwords.words('indonesian') + punctuation + ['rt', 'via']

terms_only = [term for term in preprocess(tweet['text'], lowercase=True) 
              if term not in stop and
              not term.startswith(('#', '@'))] 

count_terms_only = Counter()
count_terms_only.update(terms_only)
print(count_terms_only.most_common(20))

[('kemarin', 1), ('8.8', 1), ('19', 1), ('kamis', 1), ('pass', 1), ('birthday', 1), ('dikirimin', 1), ('ayang', 1), ('mbeb', 1), ('wkwkkw', 1), ('😛', 1), ('https://t.co/mqqtjiz5ge', 1)]


In [24]:
pip install vincent

  Stored in directory: C:\Users\ICHAL\AppData\Local\pip\Cache\wheels\4c\0d\8a\65f34c765c6094a71cce3e42a49a26533eef61695e9b505f03
Successfully built vincent
Note: you may need to restart the kernel to use updated packages.


In [25]:
import vincent
 
word_freq = count_terms_only.most_common(20)
labels, freq = zip(*word_freq)
data = {'data': freq, 'x': labels}
bar = vincent.Bar(data, iter_idx='x')
bar.to_json('term_freq.json')

In [26]:
bar.to_json('term_freq.json', html_out=True, html_path='chart.html')

In [31]:
import pandas
import json
 
dates_ITAvWAL = []
# f is the file pointer to the JSON data set
fname = 'tweetcrawled.json'
with open(fname, 'r') as f:
    a = 0
    for line in f:
        a +=1
        if a%2!=0:
            tweet = json.loads(line)
            # let's focus on hashtags only at the moment
            terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#')]
            # track when the hashtag is mentioned
            if '#kamisbucin' in terms_hash:
                dates_ITAvWAL.append(tweet['created_at'])
 
# a list of "1" to count the hashtags
ones = [1]*len(dates_ITAvWAL)
# the index of the series
idx = pandas.DatetimeIndex(dates_ITAvWAL)
# the actual series (at series of 1s for the moment)
ITAvWAL = pandas.Series(ones, index=idx)
 
# Resampling / bucketing
per_minute = ITAvWAL.resample('1Min', how='sum').fillna(0)

C:\Users\ICHAL\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()


In [32]:
time_chart = vincent.Line(ITAvWAL)
time_chart.axis_titles(x='Time', y='Freq')
time_chart.to_json('time_chart.json')